In [ ]:
import pyemu
import conda_scripts.load_datasets as lsd
import os
import pandas as pd
import numpy as np
import conda_scripts
import forward_run
import matplotlib.pyplot as plt
import conda_scripts.plot_help as ph
from conda_scripts import sv_budget
import matplotlib as mpl
import numpy as np
import shutil
import geopandas as gpd
import flopy.utils.binaryfile as bf
import flopy
import helpers

In [ ]:
workspace = lsd.model_info.get_mod('son', True)
workspace

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
ml = conda_scripts.sv_budget.load_sv_model.get_model(workspace)

In [ ]:
z = helpers.get_zones(ml)

zeros = np.zeros((ml.nrow, ml.ncol), dtype = int)

zeros[z.loc[:,'row']-1, z.loc[:,'column']-1] = z.zone

In [ ]:
    znames = z.set_index('zone').drop_duplicates('name').loc[:,'name'].to_dict()
    
    aliases = {lay*10+zint:f"{lay}_{znames[zint]}"  for zint in znames.keys() for lay in np.arange(0, 6)  }
    
    aliases

In [ ]:

def get_zone_bud_by_lay_zone(workspace):
    '''
    process zone budget of sv modflow output
    '''

    start_datetime_df = '12/1/1969'
    print(f'the start date time for the zone budget dataframe is {start_datetime_df}')

    zones_2020 = np.zeros([6, 275, 85], dtype=int)

    z = helpers.get_zones(ml)
    zeros = np.zeros((ml.nrow, ml.ncol), dtype = int)
    zeros[z.loc[:,'row']-1, z.loc[:,'column']-1] = z.zone
    
    #todo fix zonation_gwbasin_lay_

    for lay in np.arange(0, 6):
        zones_2020[lay, :, :] = lay*10
        zones_2020[lay, :, :] = zones_2020[lay, :, :] + zeros

    znames = z.set_index('zone').drop_duplicates('name').loc[:,'name'].to_dict()
    
    aliases = {lay*10+zint:f"{lay}_{znames[zint]}"  for zint in znames.keys() for lay in np.arange(0, 6)  }
    
    aliases

    cb_f = os.path.join(workspace, 'output', 'sv_model_grid_6layers.cbb')
    cbb = bf.CellBudgetFile(cb_f, verbose=False)

    # allones = np.ones((ml.dis.nrow,ml.dis.ncol),dtype = int)
    zb_whole = flopy.utils.ZoneBudget(cbb, zones_2020, aliases=aliases, totim = cbb.get_times())
    zb_df = zb_whole.get_dataframes(start_datetime=start_datetime_df, timeunit='D').multiply(1 / 43560.)

    wy = forward_run.water_year(zb_df.index.get_level_values(0))
    zb_df.loc[:, 'Water Year'] = pd.to_datetime(wy, format='%Y')
    zb_df = zb_df.set_index('Water Year', append=True)
    days = zb_df.index.get_level_values(0).daysinmonth
    zb_df.loc[:, 'Days'] = days
    zb_df = zb_df.multiply(zb_df.Days, axis='index')
    zb_df.index = zb_df.index.droplevel(0)
    zb_df = zb_df.groupby(['Water Year', 'name']).sum()

    # zb_df = zb_df.unstack().to_frame('term')

    # out_file = os.path.join(workspace, 'output', 'zbud.csv')
    # zb_df.to_csv(out_file)

    return zb_df

df = get_zone_bud_by_lay_zone(workspace)

In [ ]:
wel = cbb.get_data(text = '        MNIWELLS', full3D  =True)

In [ ]:
masked_arrays = wel

In [ ]:
import numpy as np
import xarray as xr
import pandas as pd

ml.update_modelgrid()
xl = ml.modelgrid.get_xcellcenters_for_layer(0)
yl = ml.modelgrid.get_ycellcenters_for_layer(0)

# Assume `masked_arrays` is your list of masked arrays with shape (6, 275, 85).
# Create a single masked array with an additional time dimension.

masked_array_combined = np.ma.stack(masked_arrays, axis=0)  # Shape: (time, 6, 275, 85)
zeros = np.ma.array(masked_array_combined, mask = np.abs(masked_array_combined.data)==0)

# Generate a time index starting from 12/1/1969, with monthly frequency.
time_index = pd.date_range(start="1969-12-01", periods=len(masked_arrays), freq="MS")

# Create an xarray DataArray.
data_array = xr.DataArray(
    data=masked_array_combined,  # The combined masked array.
    dims=("time", "layer", "y", "x"),

        coords=dict(
            time= time_index,
            layer= np.arange(1, 7),  # Assuming layers are numbered 1 to 6.
        x=([ "y",'x'], xl),
        y=([ "y",'x'], yl),
        # time=time,
        # reference_time=reference_time,
        ),
    
    # coords={
    #     "time": time_index,
    #     "layer": np.arange(1, 7),  # Assuming layers are numbered 1 to 6.
    #     "row": np.arange(275),
    #     "column": np.arange(85),
    # },
    name="Pumping",  # Optional name for the data array.
)

# Optionally convert to Dataset if needed.
dataset = data_array.to_dataset()



In [ ]:
summed_layers.sel(time=slice("2000-06-01", "2018-06-10")).resample(time='1Y').mean().sel(time = '2018')

In [ ]:
s.where(s!=0).to_dataframe().sort_values('masked_data').dropna()*28/(43560)

In [ ]:
# Assuming `data_array` is the xarray DataArray created previously.

# Filter to include only layers > 1 (layers start from 2 to 6).
filtered_layers = data_array.sel(layer=slice(2, 6))

# Sum across the layer dimension.
summed_layers = filtered_layers.sum(dim="layer")
ax = sv_budget.sv_budget.sv_mod_map(simple  = True)
s= summed_layers.sel(time=slice("2000-06-01", "2018-06-10")).resample(time='1Y').sum().sel(time = '2017')
s.where(s!=0).plot(vmax = -1,ax = ax ,   cmap="jet",  # Choose a colormap
    cbar_kwargs={"label": "Summed Values",'shrink':.5},  # Customize colorbar label
)

# plot summary

In [ ]:
ax = df.query("name=='TO_FARM_WELLS'").drop(columns = 'Days').groupby(level = 0).sum().sum(axis = 1).plot(drawstyle="steps-post",label = 'Total',legend = True)


ax.set_title('Total AG')
ax.grid(True)

In [ ]:

ax = df.query("name=='TO_RURWELLS' or name=='TO_MNIWELLS' ").drop(columns = 'Days').groupby(level = 0).sum().sum(axis = 1).plot(drawstyle="steps-post",label = 'Total',legend = True)

df.query("name=='TO_RURWELLS' or name=='TO_MNIWELLS' ").drop(columns = 'Days').groupby(level = [0,1]).sum().sum(axis = 1).to_frame('Q').unstack().droplevel(0,1).plot(drawstyle = "steps-post", ax = ax)
ax.set_title('Total Rural and M&I')
ax.grid(True)

In [ ]:
from shapely.geometry import Polygon

p = Polygon(ml.modelgrid.get_cell_vertices(i = 104, j = 29))


In [ ]:
ax = sv_budget.sv_budget.sv_mod_map(simple  = True)

gpd.GeoSeries(p,crs = 2226).geometry.centroid.plot(ax = ax, )

In [ ]:
df.filter(regex = "1|2").query("name=='TO_RURWELLS' or name=='TO_MNIWELLS' or name=='TO_FARM_WELLS'")

In [ ]:
df.filter(regex = "1|2").query("name=='TO_RURWELLS' or name=='TO_MNIWELLS' or name=='TO_FARM_WELLS'").sum(axis = 1).to_frame('AguaCal').unstack().plot(drawstyle = 'steps-post',title = 'Agua Caliente Pumping')

In [ ]:
df.filter(regex = 'AguaCal').filter(regex = "4|5").query("name=='TO_RURWELLS' or name=='TO_MNIWELLS' or name=='TO_FARM_WELLS'").sum(axis = 1).to_frame('AguaCal').unstack().plot()

In [ ]:
df.filter(regex = 'AguaCal').filter(regex = "2|3|4|5").query("name=='TO_RURWELLS' or name=='TO_MNIWELLS' or name=='TO_FARM_WELLS'").sum(axis = 1).to_frame('AguaCal').unstack().plot()

In [ ]:
[x for x in df.index.get_level_values(1).unique() if 'FARM' in x]